# Using COVID19API and Alpha Vantage Data to create a market analysis

In [1]:
#libraries used

import requests
import pandas as pd
import datetime

In [2]:
##api link for get use
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest

##api link for country codename
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#7934d316-f751-4914-9909-39f1901caeb8

##function for pulling new covid cases from the API

def newCasesPerDayFunc(country):
#setting variables for requests
    newCasesPerDay = []
    #country = "japan" ##doesn't work for united-states, united-kingdom,
    url = "https://api.covid19api.com/country/"+country+"/status/confirmed"

    fromdate = "2020-02-01T00:00:00Z"
    todate = "2020-07-13T00:00:00Z"
    payload = {}
    headers= {}
    params = {"from":fromdate, "to":todate}

    #response and json form of response
    response = requests.request("GET", url, headers=headers, data = payload, params=params)
    dataParse = response.json()

    def dailyCasesChange(tdayCases, ydayCases):
        dailyChange = tdayCases - ydayCases
        return dailyChange

    #loop to find data we want
    listLength = len(dataParse)
    for x in range(0, listLength):
        
        #if statement: if not day 1 do the covid cases change calculation
        if x > 0:
            dailyCountryProfile = dataParse[x]
            todaysCases = dailyCountryProfile['Cases']

            ydayCountryProfile = dataParse[x-1]
            ydayCases=ydayCountryProfile['Cases']

            newCasesToday = dailyCasesChange(todaysCases, ydayCases)
            newCasesPerDay.append(newCasesToday)

    return(newCasesPerDay)

In [3]:
startDate = '2020-02-01' #variable for start date of covid cases analysis

#list of all countries daily case data
japanNewCases = newCasesPerDayFunc("japan")
brazilNewCases = newCasesPerDayFunc("brazil")
italyNewCases = newCasesPerDayFunc("italy")
skNewCases = newCasesPerDayFunc("south-korea")

##Generates a list of all dates
datelist = pd.date_range(startDate, periods=len(japanNewCases)).tolist()

In [35]:
newDateList = []

#loop to change the date formats for table
for item in datelist:
    #print(item)
    #print(type(item))
    newDateFormat = item.strftime("%Y-%m-%d")
    newDateItem = datetime.datetime.strptime(newDateFormat, '%Y-%m-%d').strftime('%d/%m/%Y')
    newDateList.append(newDateItem)
print(newDateList)

['01/02/2020', '02/02/2020', '03/02/2020', '04/02/2020', '05/02/2020', '06/02/2020', '07/02/2020', '08/02/2020', '09/02/2020', '10/02/2020', '11/02/2020', '12/02/2020', '13/02/2020', '14/02/2020', '15/02/2020', '16/02/2020', '17/02/2020', '18/02/2020', '19/02/2020', '20/02/2020', '21/02/2020', '22/02/2020', '23/02/2020', '24/02/2020', '25/02/2020', '26/02/2020', '27/02/2020', '28/02/2020', '29/02/2020', '01/03/2020', '02/03/2020', '03/03/2020', '04/03/2020', '05/03/2020', '06/03/2020', '07/03/2020', '08/03/2020', '09/03/2020', '10/03/2020', '11/03/2020', '12/03/2020', '13/03/2020', '14/03/2020', '15/03/2020', '16/03/2020', '17/03/2020', '18/03/2020', '19/03/2020', '20/03/2020', '21/03/2020', '22/03/2020', '23/03/2020', '24/03/2020', '25/03/2020', '26/03/2020', '27/03/2020', '28/03/2020', '29/03/2020', '30/03/2020', '31/03/2020', '01/04/2020', '02/04/2020', '03/04/2020', '04/04/2020', '05/04/2020', '06/04/2020', '07/04/2020', '08/04/2020', '09/04/2020', '10/04/2020', '11/04/2020', '12/0

In [36]:
#Create DF based on a dict of lists
covidCasesDict = {'japanCases':japanNewCases, 'brazilCases': brazilNewCases,
                  'italyCases':italyNewCases, 'skCases':skNewCases}

casesDF = pd.DataFrame(covidCasesDict, index = pd.to_datetime(newDateList))

casesDF.head()

,japanCases,brazilCases,italyCases,skCases
2020-01-02,0,0,0,3
2020-02-02,0,0,0,0
2020-03-02,2,0,0,1
2020-04-02,1,0,0,3
2020-05-02,0,0,0,4


In [15]:
casesDF.tail()

,japanCases,brazilCases,italyCases,skCases
08/07/2020,356,42619,214,45
09/07/2020,427,45048,276,35
10/07/2020,386,39023,188,44
11/07/2020,411,24831,234,62
12/07/2020,284,20286,169,33


In [9]:
#Used for pulling the data
import urllib.request 
import json
#Used for formatting the data
import pandas as pd
#Used for data visualisation
import matplotlib.pyplot as plt
from matplotlib import style

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [10]:
#Returns a pandaDF of the closing stock prices in the last x amount of days
def getPriceOfStock(companySymbol, numOfDays): 
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol="+companySymbol+"&outputsize=full&apikey=Y11HUWNU7HM58RRL"
    json_obj = urllib.request.urlopen(url)
    #Finds the data from the url
    data = json.load(json_obj)
    #Choosing the revelant key from the data dictionary
    dates = data["Time Series (Daily)"] 
    #Initializing lists to store the data
    final_prices = [] 
    calendar = []
    counter = 0
    #For-loop used to run through every trading day 
    for day in dates: 
        counter += 1
        #Limits the amount of data using the number of days wanted
        if counter > numOfDays: 
            break
        #Gets the stock's price - This will include Open, High, Low, Closed prices
        prices = dates[day]
        #Choosing the stock's closed price
        priceNum = float(prices['4. close']) 
        final_prices.append(priceNum)
        calendar.append(day)
    #Dictionary containing the company and it's stock prices 
    final_prices.reverse()
    calendar.reverse()
    stockDict = {companySymbol : final_prices} #CHANGE
    #Formatting the pulled data to a Panda DataFrame
    df = pd.DataFrame(data = stockDict, index = pd.to_datetime(calendar))
    return df

In [11]:
#NOTE: Alpha Vantage only allows 5 API calls a minute, 500 a day -> Can only have an input size of list size 5
#Function to format (multiple) stocks into a PandaDF and alligns the dates
def bundleStockPrices(arrayOfCompanySymbols, numOfDays):
    largeTable = getPriceOfStock(arrayOfCompanySymbols[0], numOfDays) 
    del arrayOfCompanySymbols[0]
    for symbol in arrayOfCompanySymbols:
        smallTable = getPriceOfStock(symbol, numOfDays)
        #Concatenating the tables together -> will provide the intersection of all the data -> will provide <= 200 day data values
        #largeTable = pd.concat([largeTable, smallTable.reindex(largeTable.index)], axis = 1)
        largeTable = pd.concat([largeTable, smallTable], axis = 1, join ='inner')
    print("Any null data: "+str(largeTable.isnull().values.any()))
    print(largeTable.head())
    print("...")
    print(largeTable.tail())
    return largeTable

In [13]:
country_etf = bundleStockPrices(['JPXN','EWZ','EWI','EWY'],200)

Any null data: False
               JPXN    EWZ    EWI    EWY
2019-09-30  61.1200  42.13  27.57  56.34
2019-10-01  60.6445  41.68  27.29  56.04
2019-10-02  59.9415  40.78  26.64  55.02
2019-10-03  60.2398  41.20  26.82  55.74
2019-10-04  60.9200  42.11  26.95  56.12
...
             JPXN    EWZ    EWI    EWY
2020-07-08  59.69  30.69  25.10  59.18
2020-07-09  59.35  30.46  24.41  58.82
2020-07-10  60.08  30.79  24.87  58.43
2020-07-13  59.84  30.00  24.61  58.52
2020-07-14  60.34  30.77  25.12  59.05


In [38]:
country_stocks_covid = pd.concat([country_etf, casesDF], axis = 1, join ='inner')
print(country_stocks_covid)

               JPXN    EWZ    EWI    EWY  japanCases  brazilCases  italyCases  \
2020-01-02  64.4500  48.41  29.90  62.17           0            0           0   
2020-01-03  63.8159  47.99  29.49  60.91          19            0         342   
2020-01-06  63.9800  47.25  29.61  60.69          50        28936         318   
2020-01-07  64.1400  47.22  29.48  61.34         217        48105         201   
2020-02-03  62.6100  44.43  28.92  58.58          20            0         466   
2020-02-04  63.7300  44.72  29.42  60.24         336         1012        4585   
2020-02-05  64.2035  44.99  29.83  60.80         200         4726        1389   
2020-02-06  64.5313  44.17  30.13  61.66          30        28633         321   
2020-02-07  64.0138  43.38  29.85  60.45         130        42223         223   
2020-02-13  63.3777  43.67  29.94  61.83          10            0           0   
2020-02-14  62.7200  43.86  29.94  62.09          11            0           0   
2020-02-18  61.7694  43.32  